In [17]:
import minsearch

In [18]:
import json

In [19]:
with open('documents.json','rt') as f_in:
    docs_raw=json.load(f_in)

In [20]:
documents=[]
for course_dict in docs_raw:
    for doc in course_dict['documents']:
        doc['course']=course_dict['course']
        documents.append(doc)

In [21]:
index = minsearch.Index(
    text_fields=['question','text','section'],
    keyword_fields=['course']
)

In [22]:
q="course already started, can i still join?"

In [23]:
index.fit(documents)

In [20]:
boost={'question':3, 'section':0.5}

results=index.search(
    query=q,
    boost_dict=boost,
    num_results=5,
    filter_dict={'course':'data-engineering-zoomcamp'}
)

In [21]:
results

[{'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
  'section': 'General course-related questions',
  'question': 'Course - Can I still join the course after the start date?',
  'course': 'data-engineering-zoomcamp'},
 {'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
  'section': 'General course-related questions',
  'question': 'Course - When will the course start?',
  'course': 'data-engineerin

from openai import OpenAI

In [26]:
from openai import OpenAI
import os

In [40]:
client=OpenAI()

In [42]:
response = client.chat.completions.create(
    model="gpt-4o",
    messages=[{'role':'user','content':q}]
)
response

ChatCompletion(id='chatcmpl-BhcPkOaiGK89cDYgAuTni0wzc5UDD', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content="If the course has already started, whether you can still join depends on several factors, including the course's policies, its format, and how far into the course it is. Here are some steps you can take:\n\n1. **Contact the Instructor or Administration**: Reach out to the instructor or the administrative office associated with the course. They can provide specific information about late enrollment policies.\n\n2. **Check Online Platforms**: If the course is offered online, check the course platform for any information regarding late registration or enrollment deadlines.\n\n3. **Assess Flexibility**: Some courses, especially those that are self-paced or online, may allow late enrollments since they offer more flexibility in terms of scheduling.\n\n4. **Catch Up on Missed Content**: If joining late is possible, inquire about how 

In [44]:
response.choices[0].message.content

"If the course has already started, whether you can still join depends on several factors, including the course's policies, its format, and how far into the course it is. Here are some steps you can take:\n\n1. **Contact the Instructor or Administration**: Reach out to the instructor or the administrative office associated with the course. They can provide specific information about late enrollment policies.\n\n2. **Check Online Platforms**: If the course is offered online, check the course platform for any information regarding late registration or enrollment deadlines.\n\n3. **Assess Flexibility**: Some courses, especially those that are self-paced or online, may allow late enrollments since they offer more flexibility in terms of scheduling.\n\n4. **Catch Up on Missed Content**: If joining late is possible, inquire about how you can catch up on missed lectures, assignments, or materials.\n\n5. **Understand the Impact**: Consider how joining late might affect your learning experience

In [48]:
prompt_template="""
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database. 
Use only the facts from the CONTEXT when answering the QUESTION.
If the CONTEXT does not contain the answer, output NONE

QUESTION: {question}

CONTEXT: {context}
""".strip()
context=""
for doc in results:
    context+=f"section:{doc['section']}\nquestion:{doc['question']}\nanswer:{doc['text']}\n\n"
prompt=prompt_template.format(question=q,context=context)

In [52]:
print(prompt)

You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database. 
Use only the facts from the CONTEXT when answering the QUESTION.
If the CONTEXT does not contain the answer, output NONE

QUESTION: course already started, can i still join?

CONTEXT: section:General course-related questions
question:Course - Can I still join the course after the start date?
answer:Yes, even if you don't register, you're still eligible to submit the homeworks.
Be aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.

section:General course-related questions
question:Course - When will the course start?
answer:The purpose of this document is to capture frequently asked technical questions
The exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1
Subscribe to course public Google Calendar (it works from Desktop only).
Register be

In [54]:
response = client.chat.completions.create(
    model="gpt-4o",
    messages=[{'role':'user','content':prompt}]
)

AttributeError: 'ChatCompletion' object has no attribute 'choice'

In [57]:
response.choices[0].message.content

'Yes, even if the course has already started, you can still join and are eligible to submit the homeworks. However, be aware of the deadlines for turning in the final projects.'

In [62]:
import minsearch
from openai import OpenAI

client=OpenAI()

def search(question):
    index = minsearch.Index(
        text_fields=['question','text','section'],
        keyword_fields=['course']
    )
    index.fit(documents)
    boost={'question':3, 'section':0.5}

    results=index.search(
        query=question,
        boost_dict=boost,
        num_results=5,
        filter_dict={'course':'data-engineering-zoomcamp'}
    )
    return results

In [63]:
query = 'How I run Kafka?'
search_results = search(query)

In [38]:
def build_prompt(query, search_results):
    prompt_template = """
        You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database. 
        Use only the facts from the CONTEXT when answering the QUESTION.
        
        QUESTION: {question}
        
        CONTEXT: 
        {context}
        """.strip()
    
    context=""
    for doc in search_results:
        context+=f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"
    prompt=prompt_template.format(question=query,context=context)
    return prompt

In [39]:
def llm(prompt):
    response = client.chat.completions.create(
        model='gpt-4o',
        messages=[{'role':'user','content':prompt}]
    )
    return response.choices[0].message.content

In [78]:
query='how do I run Kafka?'
def rag(query):
    search_results=search(query)
    prompt=build_prompt(query,search_results)
    response=llm(prompt)
    return response

In [79]:
rag('the course already started, can I still join?')

"Yes, you can still join the course even after it has started. You are eligible to submit the homework, but keep in mind that there are deadlines for submitting the final projects, so it's best not to leave everything to the last minute."

In [11]:
from elasticsearch import Elasticsearch

In [12]:
es_client=Elasticsearch('http://localhost:9200')

In [13]:
es_client.info()

ObjectApiResponse({'name': '3d9c0716ed30', 'cluster_name': 'docker-cluster', 'cluster_uuid': '97nHsS8VQDKrjFivYErthg', 'version': {'number': '9.0.1', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': '73f7594ea00db50aa7e941e151a5b3985f01e364', 'build_date': '2025-04-30T10:07:41.393025990Z', 'build_snapshot': False, 'lucene_version': '10.1.0', 'minimum_wire_compatibility_version': '8.18.0', 'minimum_index_compatibility_version': '8.0.0'}, 'tagline': 'You Know, for Search'})

In [15]:
index_settings={
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} 
        }
    }
}

index_name='course-questions'

In [16]:
es_client.indices.create(index=index_name, body=index_settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'course-questions'})

In [17]:
from tqdm.auto import tqdm

In [18]:
import json
with open('documents.json','rt') as f_in:
    docs_raw=json.load(f_in)

documents=[]
for course_dic in docs_raw:
    for doc in course_dic['documents']:
        doc['course']=course_dic['course']
        documents.append(doc)

In [19]:
for doc in tqdm(documents):
    es_client.index(index=index_name, document=doc)

  0%|          | 0/948 [00:00<?, ?it/s]

In [36]:
query="I have just discovered the course, Can I join it?"
def elastic_search(query):
    search_query={
        "size": 5,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^3", "text", "section"],
                        "type": "best_fields"
                    }
                },
                "filter": {
                    "term": {
                        "course": "data-engineering-zoomcamp"
                    }
                }
            }
        }
    }
    
    response=es_client.search(index=index_name, body=search_query)
    results=[]
    for text in response['hits']['hits']:
        results.append(text['_source'])
    return results

In [37]:
elastic_search(query)

[{'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
  'section': 'General course-related questions',
  'question': 'Course - Can I still join the course after the start date?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.\nYou can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final capstone project.',
  'section': 'General course-related questions',
  'question': 'Course - Can I follow the course after it finishes?',
  'course': 'data-engineering-zoomcamp'},
 {'text': "You don't need it. You're accepted. You can also just start learning and submitting homework without registering. It is not checked agains

In [45]:
from openai import OpenAI
import os
client=OpenAI()
query='how do I run Kafka?'
def rag(query):
    search_results=elastic_search(query)
    prompt=build_prompt(query,search_results)
    response=llm(prompt)
    return response

In [46]:
rag(query)

'To run Kafka, you can execute the following command in the terminal from your project directory for running a Java Kafka Producer:\n\n```bash\njava -cp build/libs/<jar_name>-1.0-SNAPSHOT.jar:out src/main/java/org/example/JsonProducer.java\n```\n\nReplace `<jar_name>` with the actual name of the jar file in your setup. This command assumes that you have already built your project and have the necessary jar files available in the specified directory.'